In [2]:
from pathlib import Path as path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
df = pd.read_csv('tour_logs_train.csv')

In [4]:
def currency_conv(df):
    df['Ticket_Price']=df['Ticket_Price'].str.replace("$","")
    df['Ticket_Price']=df['Ticket_Price'].str.replace("USD","")
    df['Ticket_Price']=df['Ticket_Price'].str.replace("Free",'0.0')
    mask = df['Ticket_Price'].str.contains('€', na=False)

    df.loc[mask, 'Ticket_Price'] = (
    df.loc[mask, 'Ticket_Price']
    .str.replace('€', '', regex=False)
    .astype(float)
    * 1.09
    )
    mask = df['Ticket_Price'].str.contains('£', na=False)

    df.loc[mask, 'Ticket_Price'] = (
    df.loc[mask, 'Ticket_Price']
    .str.replace('£', '', regex=False)
    .astype(float)
    * 1.09
    )

def timedate():
    mask1 = df['Show_DateTime'].str.contains(
    r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}$',
    na=False
)

    mask2 = df['Show_DateTime'].str.contains(
    r'^\d{2}-\d{2}-\d{4}\s\d{2}:\d{2}$',
    na=False
)

    mask3 = df['Show_DateTime'].str.contains(
    r'^\d{2}/\d{2}/\d{4}\s\d{2}:\d{2}\s[AP]M$',
    na=False
)

    mask4 = df['Show_DateTime'].str.contains(
    r'^[A-Za-z]+\s\d{1,2},\s\d{4}$',
    na=False
)
    valid_mask = mask1|mask2|mask3|mask4
    df.loc[valid_mask, 'Show_DateTime'] = pd.to_datetime(
    df.loc[valid_mask, 'Show_DateTime'],
    errors='coerce',
    dayfirst=True
)


def price(df):
    

# 1. Force the entire column to string type to avoid 'NaN' on numeric rows
    df['Ticket_Price'] = df['Ticket_Price'].astype(str).str.strip()

# 2. Extract Normal Price (the first number in the string)
# [\d.]+ handles any number of digits and dots (like 53.2901)
    df['Normal_price'] = df['Ticket_Price'].str.extract(r'^([\d.]+)')[0]

# 3. Extract VIP Price (number following 'VIP:')
    df['VIP_price'] = df['Ticket_Price'].str.extract(r'VIP:\s*([\d.]+)')[0]

# 4. Fill missing VIP prices with the Normal price
    df['VIP_price'] = df['VIP_price'].fillna(df['Normal_price'])

# 5. Convert both to numeric so you can do math later
    df['Normal_price'] = pd.to_numeric(df['Normal_price'], errors='coerce')
    df['VIP_price'] = pd.to_numeric(df['VIP_price'], errors='coerce')


In [7]:
currency_conv(df)
timedate()
price(df)


In [8]:
df.dtypes

Gig_ID                    object
Venue_ID                  object
Show_DateTime             object
Day_of_Week                int64
Volume_Level             float64
Ticket_Price              object
Crowd_Size               float64
Opener_Rating              int64
Weather                   object
Moon_Phase                object
Band_Outfit               object
Merch_Sales_Post_Show      int64
Crowd_Energy             float64
Normal_price             float64
VIP_price                float64
dtype: object